In [1]:
pip install jieba

Using legacy 'setup.py install' for jieba, since package 'wheel' is not installed.
    Running setup.py install for jieba: started
    Running setup.py install for jieba: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\babe9\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://www.ptt.cc/bbs/C_Chat/index.html"

for i in range(50): 
 
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    rents = soup.find_all("div","r-ent")
    u = soup.select("div.btn-group.btn-group-paging a")
    url = "https://www.ptt.cc"+ u[1]["href"]  #上一頁的網址

    titles = [r.find('div',"title").text.strip() for r in rents]
    dates = [d.find('div',"meta").find("div","date").text.strip() for d in rents]

    #存成dataframe
    df = pd.DataFrame()  #暫存當頁資料

    df=pd.DataFrame(
    {
        '標題': titles,
        "日期": dates,
    })
    

In [65]:
df.head(5)

,標題,日期
0,Fw: [討論] PUMA X Pokemon 聯名開箱,11/15
1,[公連] 霸瞳的現實模樣(爆雷,11/15
2,[死神] 日番谷怎麼不選亂菊？,11/15
3,[Vtub] 拉普大人：啊啊啊啊啊,11/15
4,Re: [公連] 霸瞳的現實模樣(爆雷,11/15


In [66]:
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
from collections import Counter

In [67]:
df['seg'] = " "
df.head()

,標題,日期,seg
0,Fw: [討論] PUMA X Pokemon 聯名開箱,11/15,
1,[公連] 霸瞳的現實模樣(爆雷,11/15,
2,[死神] 日番谷怎麼不選亂菊？,11/15,
3,[Vtub] 拉普大人：啊啊啊啊啊,11/15,
4,Re: [公連] 霸瞳的現實模樣(爆雷,11/15,


In [68]:
jieba.analyse.set_stop_words('stopwords.txt')

index = 0
for documents in df["標題"]:
    seg_list = jieba.analyse.extract_tags(documents, 150)
    seg_list = jieba.lcut(documents)
    df['seg'].iloc[index] = seg_list
    index += 1

In [69]:
df["seg"]

0     [Fw, :,  , [, 討論, ],  , PUMA,  , X,  , Pokemon...
1                   [[, 公連, ],  , 霸瞳, 的, 現實, 模樣, (, 爆雷]
2                   [[, 死神, ],  , 日番, 谷, 怎麼, 不選亂, 菊, ？]
3                 [[, Vtub, ],  , 拉普, 大人, ：, 啊啊啊, 啊, 啊]
4         [Re, :,  , [, 公連, ],  , 霸瞳, 的, 現實, 模樣, (, 爆雷]
5         [[, 閒聊, ],  , 火箭, 隊才, 是, 小智, 真正, 的, 夥伴, 吧, ？]
6     [Re, :,  , [, 死神, ],  , 優哈, 巴哈, 對山, 老頭, 不錯, 吧, ？]
7                 [[, 色色, ],  , 本季, 不德, 屌, 打, 彼得, 吧, ？]
8     [Re, :,  , [, 情報, ],  , 宮本侑芽, ，, 因身體, 不, 適將, 暫...
9       [[, 亂馬, ],  , 良牙, X, 女亂, 馬, 的, 本子, 為, 啥, 這麼, 多]
10        [[, 閒聊, ],  , 大木, 博士, 其實, 對, 家庭, 很, 盡責, 吧, ？]
11      [[, 閒聊, ],  , 我, 的, 妻子, 沒有, 感情, /, 第, 24, 話, /]
12                               [[, 閒聊, ],  , 美少女, 繪師]
13    [[, 死神, ],  , 隊長, 升, 等, 考試, 要, 請, 其他, 隊長, 吃, 宴...
14    [[, 閒聊, ],  , 現實, 有, 那種, 綁, 人, 到, 車上, 的, 綁, 架,...
15    [Re, :,  , [, 亂馬, ],  , 良牙, X, 女亂, 馬, 的, 本子, 為...
16    [[, 閒聊, ],  , 墊, 肩能, 幹, 嘛, ?,  , (, EX, ., 七龍珠...
17    [Re, :,  , [, 同, 人, ],  , nayuta,  , 青, の,

In [70]:
u = 0
words = []
for i in df['seg']:
    for word in df["seg"][u]:
        words.append(word)
    u = u+1

In [71]:
show_key = pd.DataFrame(Counter(words).keys())
show_values = pd.DataFrame(Counter(words).values())

In [72]:
df1 = [show_key, show_values]
df1

[      0
 0    Fw
 1     :
 2      
 3     [
 4    討論
 ..   ..
 138   跑
 139   去
 140   做
 141  遊戲
 142   好
 
 [143 rows x 1 columns],
       0
 0     1
 1     6
 2    36
 3    20
 4     1
 ..   ..
 138   1
 139   1
 140   1
 141   1
 142   1
 
 [143 rows x 1 columns]]

In [74]:
result = pd.concat(df1, axis=1)
result.columns = ["有提到的單字", "次數"]
result

,有提到的單字,次數
0,Fw,1
1,:,6
2,,36
3,[,20
4,討論,1
...,...,...
138,跑,1
139,去,1
140,做,1
141,遊戲,1
